### Importar Arquivos e Bibliotecas

In [103]:
#Biblotecas
import pandas as pd
import pathlib as pl
import win32com.client as win32
import pythoncom

In [104]:
#Importar Base de Dados
emails = pd.read_excel(r'Bases de Dados\Emails.xlsx')
lojas = pd.read_csv(r'Bases de Dados\Lojas.csv', encoding='latin1', sep=';')
vendas = pd.read_excel(r'Bases de Dados\Vendas.xlsx')

In [105]:
#Incluir a loja na tabela de vendas
vendas = vendas.merge(lojas, on='ID Loja')

In [106]:
#Criar uma tabela para cada loja
dicionario_lojas = {}
for loja in lojas['Loja']:
    dicionario_lojas[loja] = vendas.loc[vendas['Loja'] == loja, :]

### Criar uma Tabela para cada Loja e Definir o dia do Indicador

In [107]:
#Definir o dia indicador pegando a maior data da tabela de vendas, deste modo, em uma nova data mais recente, o dia é redefinido
dia_indicador = vendas['Data'].max()

### Salvar a planilha na pasta de backup

In [108]:
#Criar pastas de Backup (e identificando se a pasta já existe)
caminho_backup = pl.Path(r'Backup Arquivos Lojas')
arquivos_backup = caminho_backup.iterdir()

lista_backup = [arquivo.name for arquivo in arquivos_backup]

for loja in dicionario_lojas:
    #Se a pasta não existe, ele cria uma nova de acordo com o shopping
    if loja not in lista_backup:
        nova_pasta = caminho_backup / loja
        nova_pasta.mkdir()

    #salvando dentro das pastas 
    nome_arquivo = '{}_{}_{}.xlsx'.format(dia_indicador.month, dia_indicador.day, loja)
    local_arquivo = caminho_backup / loja / nome_arquivo
    dicionario_lojas[loja].to_excel(local_arquivo)

### Automatizar a criação de tabelas e o envio para cada email

In [109]:
#Definir metas
meta_faturamento_dia = 1000
meta_faturamento_ano = 1_650_000
meta_produtos_dia = 4
meta_produtos_ano = 120
meta_ticket_dia = 500
meta_ticket_ano = 500

In [110]:
pythoncom.CoInitialize()
for loja in dicionario_lojas:
    vendas_loja = dicionario_lojas[loja] #Pegar apenas a loja atual
    vendas_loja_dia = vendas_loja.loc[vendas_loja['Data'] == dia_indicador, :] #Para filtrar um unico dia 

    #faturamento
    faturamento_ano = vendas_loja['Valor Final'].sum()
    faturamento_dia = vendas_loja_dia['Valor Final'].sum()

    #diversidade de produtos
    qtnd_produtos_ano = len(vendas_loja['Produto'].unique())
    qtnd_produtos_dia = len(vendas_loja_dia['Produto'].unique())

    #ticket médio
    valor_venda = vendas_loja.groupby('Código Venda').sum(numeric_only=True)
    ticket_medio_ano = valor_venda['Valor Final'].mean()
    valor_venda_dia = vendas_loja_dia.groupby('Código Venda').sum(numeric_only=True)
    ticket_medio_dia = valor_venda_dia['Valor Final'].mean()


    #enviar email
    outlook = win32.Dispatch('outlook.application')

    nome = emails.loc[emails['Loja'] == loja, 'Gerente'].values[0]
    mail = outlook.CreateItem(0)
    mail.To = emails.loc[emails['Loja'] == loja, 'E-mail'].values[0]
    mail.Subject = f'OnePage dia {dia_indicador.day}/{dia_indicador.month} - Loja {loja}'

    #Ajuste de cores para indicar as metas
    if faturamento_dia >= meta_faturamento_dia:
        cor_fat_dia = 'green'
    else:
        cor_fat_dia = 'red'

    if faturamento_ano >= meta_faturamento_ano:
        cor_fat_ano = 'green'
    else:
        cor_fat_ano = 'red'

    if qtnd_produtos_dia >= meta_produtos_dia:
        cor_qtde_dia = 'green'
    else:
        cor_qtde_dia = 'red'

    if qtnd_produtos_ano >= meta_produtos_ano:
        cor_qtde_ano = 'green'
    else:
        cor_qtde_ano = 'red'

    if ticket_medio_dia >= meta_ticket_dia:
        cor_ticket_dia = 'green'
    else:
        cor_ticket_dia = 'red'

    if ticket_medio_ano >= meta_ticket_ano:
        cor_ticket_ano = 'green'
    else:
        cor_ticket_ano = 'red'


    mail.HTMLBody = f'''
    <p>Bom dia, {nome}</p>

    <p>O resultado de ontem <strong>({dia_indicador.day}/{dia_indicador.month})</strong> da <strong>Loja {loja}</strong> foi:</p>

    <table>
    <tr>
        <th>Indicador</th>
        <th>Valor Dia</th>
        <th>Meta Dia</th>
        <th>Cenário Dia</th>
    </tr>
    <tr>
        <td>Faturamento</td>
        <td style="text-align: center">R${faturamento_dia:.2f}</td>
        <td style="text-align: center">R${meta_faturamento_dia:.2f}</td>
        <td style="text-align: center"><font color="{cor_fat_dia}">◙</th>
    </tr>
    <tr>
        <td>Diversidade de Produtos</td>
        <td style="text-align: center">{qtnd_produtos_dia}</td>
        <td style="text-align: center">{meta_produtos_dia}</td>
        <td style="text-align: center"><font color="{cor_qtde_dia}">◙</th>
    </tr>
    <tr>
        <td>Titcke Médio</td>
        <td style="text-align: center">R${ticket_medio_dia:.2f}</td>
        <td style="text-align: center">R${meta_ticket_dia:.2f}</td>
        <td style="text-align: center"><font color="{cor_ticket_dia}">◙</th>
    </tr>
    </table>
    <br>
    <table>
    <tr>
        <th>Indicador</th>
        <th>Valor Ano</th>
        <th>Meta Ano</th>
        <th>Cenário Ano</th>
    </tr>
    <tr>
        <td>Faturamento</td>
        <td style="text-align: center">R${faturamento_ano:.2f}</td>
        <td style="text-align: center">R${meta_faturamento_ano:.2f}</td>
        <td style="text-align: center"><font color="{cor_fat_ano}">◙</th>
    </tr>
    <tr>
        <td>Diversidade de Produtos</td>
        <td style="text-align: center">{qtnd_produtos_ano}</td>
        <td style="text-align: center">{meta_produtos_ano}</td>
        <td style="text-align: center"><font color="{cor_qtde_ano}">◙</th>
    </tr>
    <tr>
        <td>Titcke Médio</td>
        <td style="text-align: center">R${ticket_medio_ano:.2f}</td>
        <td style="text-align: center">R${meta_ticket_ano:.2f}</td>
        <td style="text-align: center"><font color="{cor_ticket_ano}">◙</th>
    </tr>
    </table>

    <p>Segue em anexo a planilha com todos os dados para mais detalhes.</p>

    <p>Qualquer dúvida estou à disposição.</p>
    <p>Att., Pablo Henrique</p>

    '''

    #anexos
    attachmet = pl.Path.cwd() / caminho_backup / loja / f'{dia_indicador.month}_{dia_indicador.day}_{loja}.xlsx'
    mail.Attachments.Add(str(attachmet))

    mail.Send()
    print(f'Email da Loja {loja} enviado.')
pythoncom.CoUninitialize()


Email da Loja Iguatemi Esplanada enviado.
Email da Loja Shopping Midway Mall enviado.
Email da Loja Norte Shopping enviado.
Email da Loja Shopping Iguatemi Fortaleza enviado.
Email da Loja Shopping União de Osasco enviado.
Email da Loja Shopping Center Interlagos enviado.
Email da Loja Rio Mar Recife enviado.
Email da Loja Salvador Shopping enviado.
Email da Loja Rio Mar Shopping Fortaleza enviado.
Email da Loja Shopping Center Leste Aricanduva enviado.
Email da Loja Ribeirão Shopping enviado.
Email da Loja Shopping Morumbi enviado.
Email da Loja Parque Dom Pedro Shopping enviado.
Email da Loja Bourbon Shopping SP enviado.
Email da Loja Palladium Shopping Curitiba enviado.
Email da Loja Passei das Águas Shopping enviado.
Email da Loja Center Shopping Uberlândia enviado.
Email da Loja Shopping Recife enviado.
Email da Loja Shopping Vila Velha enviado.
Email da Loja Shopping SP Market enviado.
Email da Loja Shopping Eldorado enviado.
Email da Loja Shopping Ibirapuera enviado.
Email da Lo

### Criar ranking para diretoria

In [114]:
faturamento_lojas = vendas.groupby('Loja')[['Loja', 'Valor Final']].sum(numeric_only=True)
faturamento_lojas_ano = faturamento_lojas.sort_values(by='Valor Final', ascending=False)

nome_arquivo = '{}_{}_Ranking Anual.xlsx'.format(dia_indicador.month, dia_indicador.day, loja)
faturamento_lojas_ano.to_excel(r'Backup Arquivos Lojas\{}'.format(nome_arquivo))

vendas_dia = vendas.loc[vendas['Data'] == dia_indicador, :]
faturamento_lojas_dia = vendas_dia.groupby('Loja')[['Loja', 'Valor Final']].sum(numeric_only=True)
faturamento_lojas_dia = faturamento_lojas_dia.sort_values(by='Valor Final', ascending=False)

nome_arquivo = '{}_{}_Ranking Dia.xlsx'.format(dia_indicador.month, dia_indicador.day, loja)
faturamento_lojas_dia.to_excel(r'Backup Arquivos Lojas\{}'.format(nome_arquivo))

### Enviar e-mail para diretoria

In [115]:
#enviar o email
outlook = win32.Dispatch('outlook.application')

mail = outlook.CreateItem(0)
mail.To = emails.loc[emails['Loja']=='Diretoria', 'E-mail'].values[0]
mail.Subject = f'Ranking Dia {dia_indicador.day}/{dia_indicador.month}'
mail.Body = f'''
Prezados, bom dia

Melhor loja do Dia em Faturamento: Loja {faturamento_lojas_dia.index[0]} com Faturamento R${faturamento_lojas_dia.iloc[0, 0]:.2f}
Pior loja do Dia em Faturamento: Loja {faturamento_lojas_dia.index[-1]} com Faturamento R${faturamento_lojas_dia.iloc[-1, 0]:.2f}

Melhor loja do Ano em Faturamento: Loja {faturamento_lojas_ano.index[0]} com Faturamento R${faturamento_lojas_ano.iloc[0, 0]:.2f}
Pior loja do Ano em Faturamento: Loja {faturamento_lojas_ano.index[-1]} com Faturamento R${faturamento_lojas_ano.iloc[-1, 0]:.2f}

Segue em anexo os rankings do ano e do dia de todas as lojas.

Qualquer dúvida estou à disposição.

Att.,
Lira
'''

# Anexos:
attachment = pl.Path.cwd() / caminho_backup / f'{dia_indicador.month}_{dia_indicador.day}_Ranking Anual.xlsx'
mail.Attachments.Add(str(attachment))
attachment  = pl.Path.cwd() / caminho_backup / f'{dia_indicador.month}_{dia_indicador.day}_Ranking Dia.xlsx'
mail.Attachments.Add(str(attachment))


mail.Send()
print('E-mail da Diretoria enviado')

E-mail da Diretoria enviado
